In [1]:
!pip install openpyxl --no-index

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/openpyxl-3.1.2+computecanada-py2.py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/et_xmlfile-1.1.0+computecanada-py3-none-any.whl (from openpyxl)


In [1]:
import pandas as pd
import numpy as np
import csv
import openpyxl

In [2]:
df = pd.read_csv("/lustre06/project/6060121/CLSA_PheWeb_shared/Original/23ME002_UdeM_SGTaliun_Baseline/23ME002_UdeM_SGTaliun_Baseline_CoPv7_Qx_PA_BS.csv", low_memory = False)


# Binary Traits
There are two types of binary variables. The ones with 0,1 and the ones with 1,2.
Note that 8,9 are used for DK,RF.

In [8]:
number_of_ones = (df['OST_MED_COM'] == 1).sum()
print(f"The number of 1s in the OST_MED_COM column is: {number_of_ones}")


The number of 1s in the OST_MED_COM column is: 1338


In [9]:
# Find columns that only contain 0 and 1 values and possibly missing values
#case: 1 control: 0
binary_columns_10 = [
    col for col in df.columns 
    if all((df[col].isin([1, 0, 8, 9])) | (df[col].isnull())) and df[col].notnull().any()]
len(binary_columns_10)

996

In [10]:
# Find columns that only contain 1 and 2 values and possibly missing values
#case: 1 control: 2
binary_columns_12 = [
    col for col in df.columns 
    if all((df[col].isin([1, 2, 8, 9])) | (df[col].isnull())) and df[col].notnull().any()]
len(binary_columns_12)

299

In [11]:
is_in_binary_columns_12 = "OST_MED_COM" in binary_columns_12

print(f"Is 'OST_MED_COM' in binary_columns_12? {is_in_binary_columns_12}")


Is 'OST_MED_COM' in binary_columns_12? False


In [36]:
#finding the overlap between these two binary groups
# Assuming binary_columns_01 and binary_columns_12 are your lists
binary_columns_01_set = set(binary_columns_10)
binary_columns_12_set = set(binary_columns_12)

# Find the overlap (intersection) between the two sets
overlap_columns = binary_columns_01_set.intersection(binary_columns_12_set)

# Convert the set back to a list if you need a list
overlap_columns_list = list(overlap_columns)

# Now overlap_columns_list contains the columns that are present in both binary_columns_01 and binary_columns_12
print(len(overlap_columns_list))
overlap_columns_list

13


['MEDI_DOSE_FRQ_35_COM',
 'COG_WRDFCTR_LG_COM',
 'STP_COL_OT_COM',
 'MEDI_DOSE_FRQ_38_COM',
 'FUL_STATUS_COM',
 'MEDI_DOSE_FRQ_37_COM',
 'COG_MAT_META_PSTATUS_COM',
 'TON_EYE_R_COM',
 'COG_WRDIMP_COM',
 'IAL_HPMED_COM',
 'COG_RDYIMP_COM',
 'FAL_DV_COM',
 'FAL_DVCTR_COM']

This gives us 1282 binary variables. note that this doesnt necassarily mean that the variable is binary. for instance, "MEDI_DOSE_FRQ_35_COM" (Dosage frequency - Medication 35) is not a binary variable but since it only has two unique values in the responses given, it counts as a binary variable in this step.

# Does it have more than 1k cases?

In [37]:
# from binary_columns_01 keep only the ones with at least 1k cases (case:1 control:0)
filtered_1k_columns_10 = [
    col for col in binary_columns_10 
    if df[col].value_counts().get(1, 0) >= 1000
]
len(filtered_1k_columns_10)

285

In [38]:
# from binary_columns_12 keep only the ones with at least 1k cases (case:1 control:2)
filtered_1k_columns_12 = [
    col for col in binary_columns_12 
    if df[col].value_counts().get(1, 0) >= 1000
]
len(filtered_1k_columns_12)

172

In [39]:
#finding the overlap between binary_columns_12 and binary_columns_10
# Assuming binary_columns_01 and binary_columns_12 are your lists
binary_columns_01_set_1k = set(filtered_1k_columns_10)
binary_columns_12_set_1k = set(filtered_1k_columns_12)

# Find the overlap (intersection) between the two sets
overlap_columns_1k = binary_columns_01_set_1k.intersection(binary_columns_12_set_1k)

# Convert the set back to a list if you need a list
overlap_columns_list_1k = list(overlap_columns_1k)

# Now overlap_columns_list contains the columns that are present in both binary_columns_01 and binary_columns_12
print(len(overlap_columns_list_1k))
overlap_columns_list_1k

1


['TON_EYE_R_COM']

This gives us 456 variables that are binary and have at least 1000 cases.

# Does it have "GWAS relevance"?

We meticulously went through phenotypes and selected the ones that had GWAS relevance. The results are stored in "Binary_good_GWAS_1000cases.txt" which will go through the next .ipynb notebook to finalize the phenotypes file.